**Deep Neural Network**




In [1]:
# Connecting the Python Code with the google drive
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [2]:
pip install shap

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 533.5/533.5 kB 6.3 MB/s eta 0:00:00


In [3]:
# Import necessary libraries
import sys
import os
import platform
import random
from math import sqrt, floor, ceil
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import itertools
from multiprocessing import Process
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import plot_tree
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error
import pickle

In [4]:
# Importing Data from google drive
#dat = pd.read_csv("/content/drive/MyDrive/Emory/MAF Lecture/SixSignals_20230125.csv.zip")      # Importing Data
dat = pd.read_csv("/content/drive/MyDrive/Emory/MAF Lecture/signals_20230125.csv")

In [5]:
dat

,yyyymm,new_issue_pct_rank,lag_ret_1_pct_rank,ret_2_12_pct_rank,cf2b_pct_rank,lag_at_pct_rank,investment_pct_rank,accruals_pct_rank,profitability_pct_rank,roe_pct_rank,roa_pct_rank,op_lev_pct_rank,leverage_pct_rank,b2m_pct_rank,e2p_pct_rank,s2p_pct_rank,cf2p_pct_rank,RET,PERMNO
0,199001,0.078984,-0.404498,-0.343837,0.154220,-0.136730,-0.336341,-0.417685,-0.066768,-0.306635,-0.321072,0.296086,-0.072321,0.435175,-0.391310,0.313576,0.437396,-0.200000,75540
1,199001,-0.203359,-0.221405,-0.466546,-0.499028,-0.222515,-0.470433,-0.479039,-0.427124,-0.499306,-0.477929,0.414908,0.469600,-0.454331,-0.478484,0.392143,-0.462382,0.122222,42876
2,199001,0.065936,-0.218073,0.355497,-0.199473,-0.248334,-0.078706,0.219184,-0.235564,-0.147557,-0.087313,0.250555,-0.457246,-0.229733,-0.152277,-0.185036,-0.197529,-0.278689,58560
3,199001,0.023737,0.434064,0.364381,0.160605,0.463493,0.037340,-0.256941,-0.037618,-0.187257,-0.165880,-0.160605,-0.006524,-0.315242,-0.175042,-0.226124,-0.052054,-0.183369,25320
4,199001,-0.203359,-0.211410,0.269711,0.217240,0.292476,-0.072043,0.104803,-0.414908,0.314964,0.083981,0.152277,0.357718,-0.076208,0.291644,0.304692,0.186979,-0.065934,29867
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1498038,202203,0.346755,-0.493363,-0.385693,-0.376844,-0.460619,0.430531,0.444690,-0.424041,-0.270059,-0.339676,0.329941,-0.328466,0.142625,-0.288348,0.101622,-0.402212,0.128668,19139
1498039,202203,0.236431,-0.007227,-0.474779,-0.263274,-0.382743,0.489233,0.407817,-0.227286,-0.079794,0.078319,0.164159,-0.398378,0.153245,-0.024041,-0.038496,-0.272714,0.095125,19137
1498040,202203,0.469764,0.073894,0.057375,-0.126991,0.437611,0.299853,0.370354,0.475074,0.220501,0.022271,-0.408112,-0.083628,-0.069764,0.186283,-0.284513,-0.148820,-0.155812,10777
1498041,202203,0.333186,-0.244100,-0.416667,-0.430236,-0.390708,0.162979,0.478319,0.219027,-0.340265,-0.367994,0.397788,-0.281563,-0.366814,-0.180383,-0.437611,-0.344100,-0.193136,19134


In [5]:
# Data Pre-processing

# Date-time Manipulations
dat["date"] = pd.to_datetime(dat["yyyymm"], format="%Y%m")                      # Informing python that "date" is a DateTime object for subsequent datetime manipulations like adding or subtracting months
dat["yr"] = dat["date"].dt.year                                                 # Extracting year
dat["month"] = dat["date"].dt.month                                             # Extracting month
dat.sort_values(by = 'date', inplace = True)                                    # Sorting dataframe by date
dat["month_num"] = (dat['date']).rank(method = "dense")                         # Assigning a rank for each month sequentially

# Calculating Mean & Adjusted Returns
grouped = pd.DataFrame();                                                       # Initializing DataFrame
grouped["mean"] = dat.groupby('yyyymm').apply(lambda x : x["RET"].mean())       # Computing monthly mean returns
dat['mean_ret'] = dat['yyyymm'].map(grouped["mean"])                            # Assigning mean returns to "dat["mean_ret"]" column
dat['adj_ret'] = dat['RET'] - dat['mean_ret']                                   # Adjusted Returns - subtract the mean so that the target is return minus average returns for all stocks.
                                                                                # Removes return fluctuation due to market returns - reduces "noise" in the target
# Printing Output
print("***********************************************************")
print("Pre-processed Dataframe containing all signals")
print("***********************************************************")
dat

***********************************************************
Pre-processed Dataframe containing all signals
***********************************************************


,yyyymm,new_issue_pct_rank,lag_ret_1_pct_rank,ret_2_12_pct_rank,cf2b_pct_rank,lag_at_pct_rank,investment_pct_rank,accruals_pct_rank,profitability_pct_rank,roe_pct_rank,...,s2p_pct_rank,cf2p_pct_rank,RET,PERMNO,date,yr,month,month_num,mean_ret,adj_ret
0,199001,0.078984,-0.404498,-0.343837,0.154220,-0.136730,-0.336341,-0.417685,-0.066768,-0.306635,...,0.313576,0.437396,-0.200000,75540,1990-01-01,1990,1,1.0,-0.049541,-0.150459
2395,199001,-0.050111,-0.147002,-0.046224,0.123126,0.245003,-0.348001,0.116185,0.293032,0.218073,...,-0.258329,0.004581,-0.088889,28266,1990-01-01,1990,1,1.0,-0.049541,-0.039348
2396,199001,0.151999,-0.014298,-0.005136,0.379373,0.432954,0.078429,-0.248334,0.133398,0.325514,...,-0.048168,0.397418,-0.126697,66130,1990-01-01,1990,1,1.0,-0.049541,-0.077156
2397,199001,-0.450722,0.456691,-0.042615,0.062188,0.062743,-0.014436,-0.193226,-0.236258,-0.112438,...,0.460716,0.478484,0.002857,53189,1990-01-01,1990,1,1.0,-0.049541,0.052398
2398,199001,-0.203359,-0.380900,0.014853,0.017074,-0.430455,0.467934,-0.126735,-0.462104,-0.156996,...,0.097862,0.209467,0.000000,85295,1990-01-01,1990,1,1.0,-0.049541,0.049541
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1495786,202203,-0.214307,-0.189381,0.293363,0.248230,0.336431,0.158555,0.156195,-0.115487,0.352655,...,-0.256490,-0.114307,-0.006052,84769,2022-03-01,2022,3,387.0,0.018761,-0.024813
1495787,202203,-0.123746,-0.209882,-0.149410,-0.028909,0.217109,-0.078761,-0.353097,0.103245,-0.343068,...,0.267994,0.057670,-0.048630,90979,2022-03-01,2022,3,387.0,0.018761,-0.067391
1495788,202203,-0.400147,-0.304572,0.024336,0.400147,-0.299853,-0.452065,-0.240560,0.455310,0.434071,...,-0.146460,0.326106,0.181667,85401,2022-03-01,2022,3,387.0,0.018761,0.162906
1495764,202203,-0.049410,-0.388053,0.101917,-0.388348,-0.373894,-0.391298,0.454130,0.252655,-0.281268,...,-0.399558,-0.344395,-0.354743,89891,2022-03-01,2022,3,387.0,0.018761,-0.373504


In [6]:
# Minor Pre-processing to extract available factors
y_column_name = 'adj_ret'                                                                                               # Target variable is adjusted returns
row_key_column_names = ['yyyymm', 'PERMNO', 'month', 'yr', 'month_num', 'RET', 'mean_ret','date']                       # Columns that are not signals ( either identifiers or target variables )
feature_column_names = [ x for x in dat.columns.values if (x not in row_key_column_names) & (x != y_column_name) ]      # Remaining columns are the varaibles that can potentailly be used as signals
print("************************************************************************************************************")
print("list of available signals/factors for Random Forest Model Input")
print(feature_column_names)
print("************************************************************************************************************")

************************************************************************************************************
list of available signals/factors for Random Forest Model Input
['new_issue_pct_rank', 'lag_ret_1_pct_rank', 'ret_2_12_pct_rank', 'cf2b_pct_rank', 'lag_at_pct_rank', 'investment_pct_rank', 'accruals_pct_rank', 'profitability_pct_rank', 'roe_pct_rank', 'roa_pct_rank', 'op_lev_pct_rank', 'leverage_pct_rank', 'b2m_pct_rank', 'e2p_pct_rank', 's2p_pct_rank', 'cf2p_pct_rank']
************************************************************************************************************


In [7]:
# Deep Neural Network Architecture

from keras.models import Sequential
from keras.layers import Dense, Dropout
from sklearn.preprocessing import StandardScaler
import pickle
import numpy as np

def deep_neural_network_rolling(train_X, train_y, epochs=2, batch_size=32, hidden_layers=[128, 128], dropout_rate=0.3):
    # Standardizing the data
    scaler = StandardScaler()
    train_X_scaled = scaler.fit_transform(train_X)

    # Splitting the data into training and validation sets (last 10% for validation)
    split_idx = int(0.9 * len(train_X_scaled))
    train_X_scaled, val_X_scaled = train_X_scaled[:split_idx], train_X_scaled[split_idx:]
    train_y, val_y = train_y[:split_idx], train_y[split_idx:]

    # Building the model
    model = Sequential()
    model.add(Dense(hidden_layers[0], input_dim=train_X.shape[1], activation='relu'))
    model.add(Dropout(dropout_rate))

    for units in hidden_layers[1:]:
        model.add(Dense(units, activation='relu'))
        model.add(Dropout(dropout_rate))

    model.add(Dense(1, activation='linear')) # Output layer

    # Compiling the model with additional metrics
    model.compile(optimizer='adam', loss='mean_squared_error', metrics=['mean_absolute_error'])

    # Training the model with verbose output and validation data
    history = model.fit(train_X_scaled, train_y, epochs=epochs, batch_size=batch_size,
                        validation_data=(val_X_scaled, val_y), verbose=1)

    # Saving the model and scaler
    scaler_path = '/content/drive/MyDrive/Emory/MAF Lecture/Deep_Learning_Code/scaler_6_factor_3.pkl'
    model_path = '/content/drive/MyDrive/Emory/MAF Lecture/Deep_Learning_Code/deep_neural_trained_model_6_factor_3.h5'
    model.save(model_path)
    with open(scaler_path, 'wb') as file:
        pickle.dump(scaler, file)

    return model, scaler, history




In [9]:
feature_column_names

['new_issue_pct_rank',
 'lag_ret_1_pct_rank',
 'ret_2_12_pct_rank',
 'cf2b_pct_rank',
 'lag_at_pct_rank',
 'investment_pct_rank',
 'accruals_pct_rank',
 'profitability_pct_rank',
 'roe_pct_rank',
 'roa_pct_rank',
 'op_lev_pct_rank',
 'leverage_pct_rank',
 'b2m_pct_rank',
 'e2p_pct_rank',
 's2p_pct_rank',
 'cf2p_pct_rank']

In [15]:

## Deep Neural Network Training
for feature in feature_column_names:
  predicted_ret_df = pd.DataFrame()
  rolling_window = 60
  maximum_month = int(dat["month_num"].max()) - rolling_window + 1

  train_dat = dat[(dat['month_num'] > 0) & (dat['month_num']<=60)]  # Selecting first 60 months for training
  reg_factors = feature_column_names
  y_column_name = 'adj_ret'
  train_X = train_dat[reg_factors]  # Replace 'reg_factors' with your features
  train_X = train_X.drop(feature, axis=1).copy()
  train_y = train_dat[y_column_name]
  deep_neural_network_rolling(train_X, train_y)

  # Deep Neural Network Testing
  maximum_month = int(dat["month_num"].max()) - rolling_window + 1
  predicted_ret_df = pd.DataFrame()
  for month in range(60, 160):     # INPUT THE MONTH NUMBER/ MONTH RANGE HERE

    # Input Training data & Test Data Parameters
    print("="*50)
    print(f"Model running for month {month}")

    # Loading Scaler
    scaler_path = '/content/drive/MyDrive/Emory/MAF Lecture/Deep_Learning_Code/scaler_6_factor_3.pkl'
    with open(scaler_path, 'rb') as file:
      scaler = pickle.load(file)

    # Extracting and Scaling Test Data
    test_dat = dat[ (dat['month_num'] == (month)) ]
    test_X = test_dat[reg_factors]
    test_X = test_X.drop(feature, axis=1).copy()

    test_X_scaled = scaler.transform(test_X)
    test_y = test_dat[y_column_name]

    # Loading Trained Model
    model_path = '/content/drive/MyDrive/Emory/MAF Lecture/Deep_Learning_Code/deep_neural_trained_model_6_factor_3.h5'
    model = tf.keras.models.load_model(model_path)

    # Predicting Returns
    predicted_ret = model.predict(test_X_scaled)

    ## Predicticted Returns Calculation
    test_dat = pd.DataFrame()
    test_dat = dat[(dat['month_num'] == month )].copy()
    test_dat.loc[:,"predicted_adj_ret"] = model.predict(test_X_scaled)
    test_dat.rename(columns = {"adj_ret" : "actual_adj_ret"},inplace = True)
    test_dat = test_dat[["PERMNO","yr","month","predicted_adj_ret", "actual_adj_ret", "RET"]]
    predicted_ret_df = pd.concat([predicted_ret_df, test_dat], axis =0)

  print("****************************************************************************")
  print("Dataframe with predicted returns")
  print("****************************************************************************")
  predicted_ret_df


  # Performance Evaluation
  # Decile formation & Predicted Returns Calculation
  predicted_ret_df['rank'] = predicted_ret_df.groupby(['yr','month'])['predicted_adj_ret'].transform(lambda x: pd.cut(x, 10, duplicates='drop',labels=False))         # Calculating Decile Ranks based on the Predicted Returns
  predicted_ret_df['decile_stocks_count'] = predicted_ret_df.groupby(['yr', 'month', 'rank'])['rank'].transform('count')

  predicted_ret_df.reset_index(inplace =True)                                                                                                                            # Reseting Index

  # Monthly Mean Portfolio Returns
  predicted_ret_df['RET'] = predicted_ret_df['RET']*100
  meanret = predicted_ret_df.groupby(['yr','month', 'rank'])['RET'].mean().to_frame()      # Calculating average return for each decile (according to b2m ratio) for each month
  meanret = meanret.unstack(level = -1).copy()                                             # Unstacking the grouped dataframe
  meanret[('RET', 'diff')] = meanret[('RET', 9)] -  meanret[('RET', 0)]                    # Calculating the long short returns of the portfolio by substracting "rank 0" avg. return from "rank 9" avg. return

  nmon = len(meanret)                                                                      # nmon in number of months
  meanret = meanret.stack(level = -1).copy()                                               # Stacking the dataframe to year-month index level

  # Overall Portfolio Returns Statistics
  global_mean = meanret.groupby('rank')['RET'].agg([np.mean, np.std])                      # mean and standard deviation of regression coefficients
  global_mean['t-stat'] =np.sqrt(nmon - 1) *  global_mean['mean']/global_mean['std']       # t-statistics calculation
  print(f"Ablated Feature : {feature}")
  print(global_mean)

Epoch 1/2
6152/6152 [==============================] - 18s 3ms/step - loss: 0.0467 - mean_absolute_error: 0.1198 - val_loss: 0.0309 - val_mean_absolute_error: 0.0997
Epoch 2/2
6152/6152 [==============================] - 17s 3ms/step - loss: 0.0449 - mean_absolute_error: 0.1160 - val_loss: 0.0307 - val_mean_absolute_error: 0.0993
Model running for month 60
  1/123 [..............................] - ETA: 5s

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


123/123 [==============================] - 0s 1ms/step
Model running for month 61
122/122 [==============================] - 0s 1ms/step
Model running for month 62
123/123 [==============================] - 0s 1ms/step
Model running for month 63
122/122 [==============================] - 0s 1ms/step
Model running for month 64
122/122 [==============================] - 0s 1ms/step
Model running for month 65
135/135 [==============================] - 0s 1ms/step
Model running for month 66
135/135 [==============================] - 0s 1ms/step
Model running for month 67
134/134 [==============================] - 0s 1ms/step
Model running for month 68
135/135 [==============================] - 0s 1ms/step
Model running for month 69
134/134 [==============================] - 0s 1ms/step
Model running for month 70
134/134 [==============================] - 0s 1ms/step
Model running for month 71
135/135 [==============================] - 0s 1ms/step
Model running for month 72
134/134 [=======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


123/123 [==============================] - 0s 1ms/step
Model running for month 61
122/122 [==============================] - 0s 1ms/step
Model running for month 62
123/123 [==============================] - 0s 1ms/step
Model running for month 63
122/122 [==============================] - 0s 1ms/step
Model running for month 64
122/122 [==============================] - 0s 1ms/step
Model running for month 65
135/135 [==============================] - 0s 1ms/step
Model running for month 66
135/135 [==============================] - 0s 1ms/step
Model running for month 67
134/134 [==============================] - 0s 1ms/step
Model running for month 68
135/135 [==============================] - 0s 1ms/step
Model running for month 69
134/134 [==============================] - 0s 1ms/step
Model running for month 70
134/134 [==============================] - 0s 1ms/step
Model running for month 71
135/135 [==============================] - 0s 1ms/step
Model running for month 72
134/134 [=======

...

123/123 [==============================] - 0s 1ms/step
Model running for month 61
122/122 [==============================] - 0s 1ms/step
Model running for month 62
123/123 [==============================] - 0s 1ms/step
Model running for month 63
122/122 [==============================] - 0s 1ms/step
Model running for month 64
122/122 [==============================] - 0s 1ms/step
Model running for month 65
135/135 [==============================] - 0s 1ms/step
Model running for month 66
135/135 [==============================] - 0s 1ms/step
Model running for month 67
134/134 [==============================] - 0s 1ms/step
Model running for month 68
135/135 [==============================] - 0s 1ms/step
Model running for month 69
134/134 [==============================] - 0s 1ms/step
Model running for month 70
134/134 [==============================] - 0s 1ms/step
Model running for month 71
135/135 [==============================] - 0s 1ms/step
Model running for month 72
134/134 [=======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


123/123 [==============================] - 0s 1ms/step
Model running for month 61
122/122 [==============================] - 0s 1ms/step
Model running for month 62
123/123 [==============================] - 0s 1ms/step
Model running for month 63
122/122 [==============================] - 0s 1ms/step
Model running for month 64
122/122 [==============================] - 0s 1ms/step
Model running for month 65
135/135 [==============================] - 0s 1ms/step
Model running for month 66
135/135 [==============================] - 0s 1ms/step
Model running for month 67
134/134 [==============================] - 0s 1ms/step
Model running for month 68
135/135 [==============================] - 0s 1ms/step
Model running for month 69
134/134 [==============================] - 0s 1ms/step
Model running for month 70
134/134 [==============================] - 0s 1ms/step
Model running for month 71
135/135 [==============================] - 0s 1ms/step
Model running for month 72
134/134 [=======

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


123/123 [==============================] - 0s 1ms/step
Model running for month 61
122/122 [==============================] - 0s 1ms/step
Model running for month 62
123/123 [==============================] - 0s 1ms/step
Model running for month 63
122/122 [==============================] - 0s 1ms/step
Model running for month 64
122/122 [==============================] - 0s 1ms/step
Model running for month 65
135/135 [==============================] - 0s 1ms/step
Model running for month 66
135/135 [==============================] - 0s 1ms/step
Model running for month 67
134/134 [==============================] - 0s 1ms/step
Model running for month 68
135/135 [==============================] - 0s 1ms/step
Model running for month 69
134/134 [==============================] - 0s 1ms/step
Model running for month 70
134/134 [==============================] - 0s 1ms/step
Model running for month 71
135/135 [==============================] - 0s 1ms/step
Model running for month 72
134/134 [=======

In [8]:
predicted_ret_df = pd.DataFrame()
rolling_window = 60
maximum_month = int(dat["month_num"].max()) - rolling_window + 1

train_dat = dat[(dat['month_num'] > 0) & (dat['month_num']<=60)]  # Selecting first 60 months for training
reg_factors = feature_column_names
y_column_name = 'adj_ret'
train_X = train_dat[reg_factors]  # Replace 'reg_factors' with your features
#train_X = train_X.drop(feature, axis=1).copy()
train_y = train_dat[y_column_name]
deep_neural_network_rolling(train_X, train_y)

# Deep Neural Network Testing
maximum_month = int(dat["month_num"].max()) - rolling_window + 1
predicted_ret_df = pd.DataFrame()
for month in range(60, 160):     # INPUT THE MONTH NUMBER/ MONTH RANGE HERE

  # Input Training data & Test Data Parameters
  print("="*50)
  print(f"Model running for month {month}")

  # Loading Scaler
  scaler_path = '/content/drive/MyDrive/Emory/MAF Lecture/Deep_Learning_Code/scaler_6_factor_3.pkl'
  with open(scaler_path, 'rb') as file:
    scaler = pickle.load(file)

  # Extracting and Scaling Test Data
  test_dat = dat[ (dat['month_num'] == (month)) ]
  test_X = test_dat[reg_factors]
  #test_X = test_X.drop(feature, axis=1).copy()

  test_X_scaled = scaler.transform(test_X)
  test_y = test_dat[y_column_name]

  # Loading Trained Model
  model_path = '/content/drive/MyDrive/Emory/MAF Lecture/Deep_Learning_Code/deep_neural_trained_model_6_factor_3.h5'
  model = tf.keras.models.load_model(model_path)

  # Predicting Returns
  predicted_ret = model.predict(test_X_scaled)

  ## Predicticted Returns Calculation
  test_dat = pd.DataFrame()
  test_dat = dat[(dat['month_num'] == month )].copy()
  test_dat.loc[:,"predicted_adj_ret"] = model.predict(test_X_scaled)
  test_dat.rename(columns = {"adj_ret" : "actual_adj_ret"},inplace = True)
  test_dat = test_dat[["PERMNO","yr","month","predicted_adj_ret", "actual_adj_ret", "RET"]]
  predicted_ret_df = pd.concat([predicted_ret_df, test_dat], axis =0)

print("****************************************************************************")
print("Dataframe with predicted returns")
print("****************************************************************************")
predicted_ret_df


# Performance Evaluation
# Decile formation & Predicted Returns Calculation
predicted_ret_df['rank'] = predicted_ret_df.groupby(['yr','month'])['predicted_adj_ret'].transform(lambda x: pd.cut(x, 10, duplicates='drop',labels=False))         # Calculating Decile Ranks based on the Predicted Returns
predicted_ret_df['decile_stocks_count'] = predicted_ret_df.groupby(['yr', 'month', 'rank'])['rank'].transform('count')

predicted_ret_df.reset_index(inplace =True)                                                                                                                            # Reseting Index

# Monthly Mean Portfolio Returns
predicted_ret_df['RET'] = predicted_ret_df['RET']*100
meanret = predicted_ret_df.groupby(['yr','month', 'rank'])['RET'].mean().to_frame()      # Calculating average return for each decile (according to b2m ratio) for each month
meanret = meanret.unstack(level = -1).copy()                                             # Unstacking the grouped dataframe
meanret[('RET', 'diff')] = meanret[('RET', 9)] -  meanret[('RET', 0)]                    # Calculating the long short returns of the portfolio by substracting "rank 0" avg. return from "rank 9" avg. return

nmon = len(meanret)                                                                      # nmon in number of months
meanret = meanret.stack(level = -1).copy()                                               # Stacking the dataframe to year-month index level

# Overall Portfolio Returns Statistics
global_mean = meanret.groupby('rank')['RET'].agg([np.mean, np.std])                      # mean and standard deviation of regression coefficients
global_mean['t-stat'] =np.sqrt(nmon - 1) *  global_mean['mean']/global_mean['std']       # t-statistics calculation
#print(f"Ablated Feature : {feature}")
print(global_mean)

Epoch 1/2
6152/6152 [==============================] - 20s 3ms/step - loss: 0.0464 - mean_absolute_error: 0.1194 - val_loss: 0.0308 - val_mean_absolute_error: 0.0996
Epoch 2/2
6152/6152 [==============================] - 17s 3ms/step - loss: 0.0449 - mean_absolute_error: 0.1159 - val_loss: 0.0308 - val_mean_absolute_error: 0.0997


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


Model running for month 60
123/123 [==============================] - 0s 1ms/step
Model running for month 61
122/122 [==============================] - 0s 1ms/step
Model running for month 62
123/123 [==============================] - 0s 1ms/step
Model running for month 63
122/122 [==============================] - 0s 1ms/step
Model running for month 64
122/122 [==============================] - 0s 1ms/step
Model running for month 65
135/135 [==============================] - 0s 1ms/step
Model running for month 66
135/135 [==============================] - 0s 1ms/step
Model running for month 67
134/134 [==============================] - 0s 1ms/step
Model running for month 68
135/135 [==============================] - 0s 1ms/step
Model running for month 69
134/134 [==============================] - 0s 1ms/step
Model running for month 70
134/134 [==============================] - 0s 1ms/step
Model running for month 71
135/135 [==============================] - 0s 1ms/step
Model running fo

NameError: ignored

In [9]:
global_mean


,mean,std,t-stat
rank,,,
0,-2.745394,12.549124,-2.176752
1,-1.482839,9.780500,-1.508518
2,0.193426,8.005360,0.240410
3,1.920744,9.775468,1.955013
4,3.500723,12.680340,2.746910
5,5.674042,16.176709,3.489956
6,8.077986,16.442963,4.888106
7,10.808542,21.304142,5.048015
8,13.247819,26.257728,5.020013
